In [1]:
# Import
import pandas as pd
from PIL import Image
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [2]:
# Path & Declaration
test_path = './dataset/test/'
train_path = './dataset/train/'
valid_path = './dataset/valid/'

types = ['Brazil', 'Canada', 'Finland', 'Japan', 'United-Kingdom', 'United_States']

In [19]:
# Preprocess
def process_data(path):
    df = pd.read_csv(path + '_classes.csv')
    result_img = []
    result_idx = []
    

    for _, row in df.iterrows():
        try:
            image_path = row['filename']
            image = Image.open(path + image_path)
            image = image.resize((224, 224)) 
            normalized_image = np.array(image) / 255.0
            label_index = np.argmax(row.iloc[1:].values)
            result_img.append(normalized_image)
            result_idx.append(label_index)
        except:
            continue

    result_idx = to_categorical(result_idx, num_classes=7)
    result_img = np.array(result_img, dtype=np.float32)
    result_idx = np.array(result_idx, dtype=np.float32)

    return tf.data.Dataset.from_tensor_slices((result_img, result_idx)).batch(16)

In [20]:
# Tensorflow Data
test_data = process_data(test_path)
train_data = process_data(train_path)
valid_data = process_data(valid_path)

In [29]:
#AlexNet Model
def AlexNet():
  inp = layers.Input((224, 224, 3))
  x = layers.Conv2D(96, 11, 4, activation='relu')(inp)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(3, 2)(x)
  x = layers.Conv2D(256, 5, 1, activation='relu')(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(3, 2)(x)
  x = layers.Conv2D(384, 3, 1, activation='relu')(x)
  x = layers.Conv2D(384, 3, 1, activation='relu')(x)
  x = layers.Conv2D(256, 3, 1, activation='relu')(x)
  x = layers.MaxPooling2D(3, 2)(x)
  x = layers.Flatten()(x)
  x = layers.Dense(4096, activation='relu')(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(4096, activation='relu')(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(7, activation='softmax')(x)

  model = Model(inputs=inp, outputs=x)

  return model

model = AlexNet()

In [30]:
# Model Compile
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="categorical_crossentropy", metrics=['accuracy'])

In [33]:
# Model Fit
es = ReduceLROnPlateau(monitor='val_loss', patience = 5, verbose=1,factor=0.5, min_lr=0.00001)
    
model.fit(train_data, batch_size=24, epochs=15, validation_data=valid_data,
          callbacks=[es])

Epoch 1/15


385/385 [==============================] - 123s 318ms/step - loss: 1.2870 - accuracy: 0.5019 - val_loss: 1.7676 - val_accuracy: 0.3120 - lr: 1.0000e-04
Epoch 2/15
385/385 [==============================] - 124s 322ms/step - loss: 1.1533 - accuracy: 0.5526 - val_loss: 2.1390 - val_accuracy: 0.2766 - lr: 1.0000e-04
Epoch 3/15
385/385 [==============================] - 123s 320ms/step - loss: 0.9848 - accuracy: 0.6265 - val_loss: 2.2201 - val_accuracy: 0.3035 - lr: 1.0000e-04
Epoch 4/15
385/385 [==============================] - 130s 337ms/step - loss: 0.8292 - accuracy: 0.6939 - val_loss: 2.6735 - val_accuracy: 0.2811 - lr: 1.0000e-04
Epoch 5/15
385/385 [==============================] - 138s 359ms/step - loss: 0.7080 - accuracy: 0.7403 - val_loss: 2.4184 - val_accuracy: 0.3457 - lr: 1.0000e-04
Epoch 6/15
385/385 [==============================] - ETA: 0s - loss: 0.5803 - accuracy: 0.7870
Epoch 6: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
385/385 [===============

In [35]:
# Save Model
model.save("model.h5")